In [1]:
from os.path import isfile
from statistics import mode

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from lifelines.utils import concordance_index
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sksurv.linear_model import CoxnetSurvivalAnalysis

# Data load

In [2]:
clients = pd.read_csv("data/clients.csv")
train = pd.read_csv("data/train.csv")
report_dates = pd.read_csv("data/report_dates.csv")
sample_submit_naive = pd.read_csv("data/sample_submit_naive.csv")
sample = pd.read_csv("data/sample_submit_naive.csv")
transactions = pd.read_csv("data/transactions.csv")

# Feature engineering from transaction history

In [3]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13075023 entries, 0 to 13075022
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           int64  
 1   mcc_code          int64  
 2   currency_rk       int64  
 3   transaction_amt   float64
 4   transaction_dttm  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 498.8+ MB


In [4]:
%%time

if isfile("data/basic_features.csv"):
    print("Read features from csv")
    features = pd.read_csv("data/basic_features.csv")
else:
    print("Create features from transaction history")
    transactions_grouped = transactions.groupby(by="user_id")
    transactions = transactions.merge(clients[["user_id", "report"]]).merge(
        report_dates
    )

    transactions["day_diff"] = (
        pd.to_datetime(transactions["report_dt"])
        - pd.to_datetime(transactions["transaction_dttm"])
    ).dt.days

    transactions["abs_tr"] = abs(transactions["transaction_amt"])
    features = transactions.groupby("user_id").agg(
        sum_tr=("transaction_amt", "sum"),
        mean_sum_tr=("transaction_amt", "mean"),
        std_sum_tr=("transaction_amt", "std"),
        count_sum_tr=("transaction_amt", "count"),
        qnt10_sum_tr=("transaction_amt", lambda x: x.quantile(0.1)),
        qnt90_sum_tr=("transaction_amt", lambda x: x.quantile(0.9)),
        sum_abs_tr=("abs_tr", "sum"),
        mean_abs_sum_tr=("abs_tr", "mean"),
        std_abs_sum_tr=("abs_tr", "std"),
        count_abs_sum_tr=("abs_tr", "count"),
        qnt10_abs_sum_tr=("abs_tr", lambda x: x.quantile(0.1)),
        qnt90_abs_sum_tr=("abs_tr", lambda x: x.quantile(0.9)),
        sum_pos_tr=("transaction_amt", lambda x: sum(x[x > 0])),
        mean_pos_sum_tr=("transaction_amt", lambda x: x[x > 0].mean()),
        std_pos_sum_tr=("transaction_amt", lambda x: x[x > 0].std()),
        count_pos_sum_tr=("transaction_amt", lambda x: x[x > 0].count()),
        qnt10_pos_sum_tr=("transaction_amt", lambda x: x[x > 0].quantile(0.1)),
        qnt90_pos_sum_tr=("transaction_amt", lambda x: x[x > 0].quantile(0.9)),
        sum_neg_tr=("transaction_amt", lambda x: sum(x[x < 0])),
        mean_neg_sum_tr=("transaction_amt", lambda x: x[x < 0].mean()),
        std_neg_sum_tr=("transaction_amt", lambda x: x[x < 0].std()),
        count_neg_sum_tr=("transaction_amt", lambda x: x[x < 0].count()),
        qnt10_neg_sum_tr=("transaction_amt", lambda x: x[x < 0].quantile(0.1)),
        qnt90_neg_sum_tr=("transaction_amt", lambda x: x[x < 0].quantile(0.9)),
        life=("day_diff", lambda x: max(x) - min(x)),
        unique_days=("day_diff", lambda x: len(set(x))),
        last_day_tr=("day_diff", "last"),
        last_day_tr_amt=("transaction_amt", "last"),
        mode_mcc_code_eq_one=("mcc_code", lambda x: mode(x) == 1),
        mode_currency_eq_one=("currency_rk", lambda x: mode(x) == 1),
    )
    features.to_csv("data/basic_features.csv")
features

Read features from csv
CPU times: total: 375 ms
Wall time: 379 ms


,user_id,sum_tr,mean_sum_tr,std_sum_tr,count_sum_tr,qnt10_sum_tr,qnt90_sum_tr,sum_abs_tr,mean_abs_sum_tr,std_abs_sum_tr,...,std_neg_sum_tr,count_neg_sum_tr,qnt10_neg_sum_tr,qnt90_neg_sum_tr,life,unique_days,last_day_tr,last_day_tr_amt,mode_mcc_code_eq_one,mode_currency_eq_one
0,3,13706.416641,1246.037876,60860.880477,11,-15144.601562,37991.929688,358510.042953,32591.822087,50371.669520,...,74126.238593,4,-112250.203906,-1090.649873,106,8,108,104011.960938,False,True
1,9,-323434.666813,-3593.718520,11797.055119,90,-4270.121582,-219.823502,323434.666813,3593.718520,11797.055119,...,11797.055119,90,-4270.121582,-219.823502,181,54,102,-4246.083008,True,True
2,13,-124717.379150,-5668.971780,23507.953991,22,-23855.722852,10937.443262,382250.747803,17375.033991,16432.403361,...,12945.866866,14,-24566.357031,-9605.339648,168,18,114,-16394.193359,False,True
3,37,-331859.599463,-1053.522538,3913.114835,315,-1940.371729,-74.076974,353337.176611,1121.705323,3894.057303,...,3891.580411,313,-1971.609570,-76.457050,179,130,104,-2030.448853,False,True
4,41,-108586.614166,-6786.663385,5251.737982,16,-12649.590820,-531.802338,108586.614166,6786.663385,5251.737982,...,5251.737982,16,-12649.590820,-531.802338,153,12,103,-6226.305176,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,562043,-29581.256115,-799.493409,2136.038913,37,-1320.301514,-70.616780,29581.256115,799.493409,2136.038913,...,2136.038913,37,-1320.301514,-70.616780,124,23,142,-142.056610,True,True
95996,562205,-40491.908630,-268.158335,590.682233,151,-697.885010,-22.832464,44429.721298,294.236565,578.049705,...,573.576098,148,-698.641840,-22.964891,178,84,102,-422.024323,False,True
95997,562312,-18537.821270,-331.032523,309.587795,56,-708.046326,-45.675522,18537.821270,331.032523,309.587795,...,309.587795,56,-708.046326,-45.675522,158,38,122,-1164.898438,True,True
95998,562721,-164004.761685,-1929.467785,4172.623511,85,-7118.363672,1811.933887,232787.089472,2738.671641,3686.236642,...,4066.990061,66,-8625.995117,-32.536149,175,55,105,-3608.912842,False,True


# Data preprocessing

In [5]:
full_merged = clients.merge(features, how="left", on="user_id", validate="one_to_one")
full_merged["isna_employee_count_nm"] = full_merged["employee_count_nm"].isna()
full_merged.replace(
    {
        "employee_count_nm": {
            "ДО 10": 1,
            "ОТ 11 ДО 30": 2,
            "ОТ 11 ДО 50": 3,
            "ОТ 31 ДО 50": 4,
            "ОТ 51 ДО 100": 5,
            "ОТ 101 ДО 500": 6,
            "ОТ 501 ДО 1000": 7,
            "БОЛЕЕ 500": 8,
            "БОЛЕЕ 1001": 9,
        }
    },
    inplace=True,
)
full_merged.fillna(0, inplace=True)
full_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96000 entries, 0 to 95999
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 96000 non-null  int64  
 1   report                  96000 non-null  int64  
 2   employee_count_nm       96000 non-null  float64
 3   bankemplstatus          96000 non-null  int64  
 4   customer_age            96000 non-null  int64  
 5   sum_tr                  96000 non-null  float64
 6   mean_sum_tr             96000 non-null  float64
 7   std_sum_tr              96000 non-null  float64
 8   count_sum_tr            96000 non-null  int64  
 9   qnt10_sum_tr            96000 non-null  float64
 10  qnt90_sum_tr            96000 non-null  float64
 11  sum_abs_tr              96000 non-null  float64
 12  mean_abs_sum_tr         96000 non-null  float64
 13  std_abs_sum_tr          96000 non-null  float64
 14  count_abs_sum_tr        96000 non-null

In [6]:
train_merged = train.merge(full_merged, how="left", on="user_id", validate="one_to_one")
train_merged

,user_id,target,time,report,employee_count_nm,bankemplstatus,customer_age,sum_tr,mean_sum_tr,std_sum_tr,...,count_neg_sum_tr,qnt10_neg_sum_tr,qnt90_neg_sum_tr,life,unique_days,last_day_tr,last_day_tr_amt,mode_mcc_code_eq_one,mode_currency_eq_one,isna_employee_count_nm
0,3,0,77,2,6.0,0,3,13706.416641,1246.037876,60860.880477,...,4,-112250.203906,-1090.649873,106,8,108,104011.960938,False,True,False
1,13,0,86,6,7.0,0,2,-124717.379150,-5668.971780,23507.953991,...,14,-24566.357031,-9605.339648,168,18,114,-16394.193359,False,True,False
2,37,0,89,5,9.0,0,2,-331859.599463,-1053.522538,3913.114835,...,313,-1971.609570,-76.457050,179,130,104,-2030.448853,False,True,False
3,41,0,57,1,6.0,0,2,-108586.614166,-6786.663385,5251.737982,...,16,-12649.590820,-531.802338,153,12,103,-6226.305176,False,True,False
4,42,0,84,12,1.0,0,3,11429.587215,193.721817,6735.085596,...,48,-2293.932056,-65.718312,181,38,102,892.215759,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63995,561824,0,91,12,0.0,0,0,-74729.259329,-504.927428,1046.917405,...,144,-961.224933,-63.462321,178,56,102,-212.558014,True,True,True
63996,562043,0,75,12,0.0,0,2,-29581.256115,-799.493409,2136.038913,...,37,-1320.301514,-70.616780,124,23,142,-142.056610,True,True,True
63997,562312,0,91,12,0.0,0,0,-18537.821270,-331.032523,309.587795,...,56,-708.046326,-45.675522,158,38,122,-1164.898438,True,True,True
63998,562721,0,29,12,0.0,0,2,-164004.761685,-1929.467785,4172.623511,...,66,-8625.995117,-32.536149,175,55,105,-3608.912842,False,True,True


# Prepare train and test sets

In [7]:
X = train_merged.drop(columns=["user_id", "time", "report"])
y = train_merged.apply(lambda df: (df["target"], df["time"]), axis=1).to_numpy(
    np.dtype([("target", "?"), ("time", "<f8")])
)
y

array([(False, 77.), (False, 86.), (False, 89.), ..., (False, 91.),
       (False, 29.), (False, 91.)],
      dtype=[('target', '?'), ('time', '<f8')])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=22, stratify=X["target"]
)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=0.2, random_state=222, stratify=X_train["target"]
# )
X_train.drop(columns=["target"], inplace=True)
X_test.drop(columns=["target"], inplace=True)
# X_val.drop(columns=["target"], inplace=True)

# Training

In [14]:
coxnet_pipe = make_pipeline(
    StandardScaler(),
    CoxnetSurvivalAnalysis(l1_ratio=0.9, alpha_min_ratio=0.0004),
)
coxnet_pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('coxnetsurvivalanalysis',
                 CoxnetSurvivalAnalysis(alpha_min_ratio=0.0004, l1_ratio=0.9))])

In [18]:
score_test = concordance_index(
    y_test[:]["time"], -coxnet_pipe.predict(X_test), y_test[:]["target"]
)
score_train = concordance_index(
    y_train[:]["time"], -coxnet_pipe.predict(X_train), y_train[:]["target"]
)
print(
    "Test concordance index={:.4}, train concordance index={:.4}.".format(
        score_test,
        score_train,
    )
)

Test concordance index=0.7094, train concordance index=0.7184.


# Submit saving

In [16]:
x_test_sample = sample.merge(
    full_merged, how="left", on="user_id", validate="one_to_one"
).drop(columns=["user_id", "predict", "report"])

y_test_predicted = coxnet_pipe.predict(x_test_sample)
y_test_predicted

array([-1.21475695,  0.36138487,  0.4203591 , ...,  0.21096396,
       -0.42541041,  0.57912692])

In [17]:
sample["predict"] = y_test_predicted
sample.to_csv("submit.csv", index=False)